In [57]:
import numpy as np
import pandas as pd

loan_file = pd.read_csv(".\\loan\\loan.csv")

C:\Users\admin\AppData\Local\Temp\ipykernel_2492\1363479270.py:4: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_file = pd.read_csv(".\\loan\\loan.csv")


Identify Blank Columns

In [58]:

loan_file.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [59]:
loan_file.shape

(39717, 111)

In [60]:
type(loan_file.isnull().sum())

filtered_series = loan_file.isnull().sum()[loan_file.isnull().sum() == 39717]
print(filtered_series.shape)
filtered_series.to_csv('cols_having_empty_header.csv',header=['Count of Row Values'],index=True)


(54,)


Remove the empty columns

In [61]:
loan_file.dropna(axis=1,how='all',inplace=True)
loan_file.shape
total_length = len(loan_file)

# loan_file.columns[]
print((loan_file.columns[(loan_file.isnull().sum()/total_length) > 0.75]).value_counts())
print((loan_file.columns[(loan_file.isnull().sum()/total_length) > 0.50]).value_counts())
print((loan_file.columns[(loan_file.isnull().sum()/total_length) > 0.25]).value_counts())

loan_file.drop(columns=loan_file.columns[(loan_file.isnull().sum()/total_length) > 0.50],inplace=True)
print(loan_file.shape)
loan_file.to_csv(".\\iteration_1.csv")

mths_since_last_record    1
next_pymnt_d              1
Name: count, dtype: int64
mths_since_last_delinq    1
mths_since_last_record    1
next_pymnt_d              1
Name: count, dtype: int64
desc                      1
mths_since_last_delinq    1
mths_since_last_record    1
next_pymnt_d              1
Name: count, dtype: int64
(39717, 54)


Now delete all columns with a single value

In [62]:
loan_file.drop(columns=['pymnt_plan','url','desc','initial_list_status','collections_12_mths_ex_med','policy_code','application_type','acc_now_delinq','chargeoff_within_12_mths','delinq_amnt','tax_liens'],inplace=True)
print(loan_file.shape)

loan_file.to_csv(".\\iteration_2.csv")


(39717, 43)


Deleting rows which are descriptive and may not contribute to identification

| Field Name | Remarks |
| -------- | -------- |
| member_id | We shall retain id for reference purpose. Assumption - member_id will be retrieved from id |
| emp_title | Descriptive. Does not seem to contribute. |
| emp_length | Descriptive. Does not seem to contribute. |
| title | Descriptive. Does not seem to contribute. |
| last_credit_pull_d | The most recent month LC pulled credit for this loan - Does not contribute towards analysis.. |
| last_pymnt_amnt |  Last total payment amount received - Does not contribute towards analysis.|
| last_pymnt_d | Last month payment was received - Does not contribute towards analysis. |
| collection_recovery_fee | post charge off collection fee - Does not contribute towards analysis. |
| earliest_cr_line | The month the borrower's earliest reported credit line was opened - Does not contribute towards analysis.|
| addr_state | The state provided by the borrower in the loan application - All the states are allowed . |
| Zip Code| The first 3 numbers of the zip code provided by the borrower in the loan application.  Removed to prevent redlining and bias from the model. |
| issue_d |  The month which the loan was funded- Does not contribute towards analysis.|
| funded_amnt |  The total amount committed to that loan at that point in time.|
| funded_amnt_inv |  The total amount committed by investors for that loan at that point in time..|
| verification_status |  Indicates if income was verified by LC, not verified, or if the income source was verified.|







In [63]:
loan_file.drop(columns=['member_id','emp_title','emp_length','title','last_credit_pull_d','last_pymnt_amnt','last_pymnt_d','collection_recovery_fee','earliest_cr_line','addr_state','zip_code','issue_d','funded_amnt','funded_amnt_inv','verification_status'],inplace=True)
print(loan_file.shape)

loan_file.to_csv(".\\iteration_3.csv")

(39717, 29)
